<a href="https://colab.research.google.com/github/josealbertorg7/speechrecognition/blob/main/Grafica_con_Datos_Estimulaciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Datos para gráfica --- Ingenieros de estimulaciones

----------------------------------------------------------------------------------------------------------------------------------------------------------------

Agrega el archivo ASCII compilando el siguiente bloque de código

In [ ]:
#SUBE EL ARCHIVO ASCII
from google.colab import files
uploaded = files.upload()

Saving ASCII Chuc-172.txt to ASCII Chuc-172.txt


Compila este siguiente bloque de código para sacar los datos y mostrar la tabla de la gráfica con datos a entregar con el cliente

In [ ]:
#Importar librerias (analisis de datos pandas, mates numpy)
import pandas as pd
import numpy as np

# Obtener nombre del archivo como cadena de texto chain caracter para poder abrirlo con open()
filename = list(uploaded.keys())[0]

# Leer .txt linea por linea sin necesidad de close(filename) de manera automatica con with
with open(filename, 'r') as file:
    lines = file.readlines()

# Enumerar la posición en donde está la fila de los nombres de las variables
for i, line in enumerate(lines):
    if "Time" in line:
        header_line_index = i
        break

# REDONDEO SLUR RATE ---- si decimal < 0.4 REDONDEA HACIA ABAJO, SI >0.6 REDONDEA HACIA ARRIBA, DECIMAL = 0.4 = 0.5, PONE 0.5
def redondeo_slurry_rate(x):
    numero = int(x)
    decimal = x - numero

    if decimal < 0.4:
        return float(numero)
    elif 0.4 <= decimal <= 0.5:
        return float(numero) + 0.5
    else:
        return float(numero + 1)

# Leer datos, usando
data = pd.read_csv(   #Convertir a tabla el ascii
    filename,
    skiprows = header_line_index + 3, #normalmente saltar primer fila y 2 mas en codigos ASCII
    sep=r'\s+', #usar espacios como separador de filas/columnas
    names=lines[header_line_index].strip().split("\t") #definir variables/nombres de columnas
)

# Quitar espacios y poner guiones bajos como separador a las variables,  en vez de slur rate, sería SLUR_RATE
data.columns = [col.strip().upper().replace(" ", "_") for col in data.columns]


# Convertir variable TIME a formato de fecha y hora, si no trae hora, eliminar pos ni pedo
data["TIME"] = pd.to_datetime(data["TIME"], format="%m:%d:%Y:%H:%M:%S", errors='coerce')
data = data.dropna(subset=["TIME"])

# Agrupar por etapa STP_NUMBER para poder hacer operaciones matematicas con esa etapa
#Objeto tipo DataFrameGroupby, aplicar alguna operacion para ver resultados
grouped = data.groupby("STP_NUMBER")


# Crear tabla con datos necesarios para entregable Grafica con datos PONER LAS VARIABLES QUE APARECEN EN TU ASCII
summary = grouped.agg({
    "SLUR_RATE": lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan, #Calcula moda para no sacar valores erroneos como .9 0 .1 extras en los gastos liquidos
    "STP_SLUR": "max",
    "N2_RATE": "max",
    "STP_N2": "max",
    "TR_PRESS": "last",#A VECES EN SANDCAT ES "TREATING_PRESSURE"
    "AN_PRESS": "last"
}).reset_index()




# Redondeo a 1 decimal donde corresponde
summary["STP_N2"] = summary["STP_N2"].round(1)
summary["STP_SLUR"] = summary["STP_SLUR"].round(1)
summary["N2_RATE"] = summary["N2_RATE"].round(1)
summary["SLUR_RATE"] = summary["SLUR_RATE"].apply(redondeo_slurry_rate) #Aplica la regla de la función que definimos para redondeo personalizado al slurry rate
#porque a veces las bombas dan el gasto un decimal mas bajo o masl alto en su caso

# Renombrar columnas (ahora sí con los nombres a como queramos ua)
summary.columns = [
    "Etapa","Gasto líquido (bpm)", "Volumen líquido (m3)",
    "Gasto N2 (m3/min)", "Volumen N2 (m3)", "TP (psi)", "TR (psi)"
]

# Mostrar tabla completa sin el índice puesto por py (index) y con los nombres a como queramos con 1 decimal
pd.set_option("display.max_rows", None)
from IPython.display import display
display(summary.style.format({
     "Gasto líquido (bpm)": "{:.1f}",
     "Volumen líquido (m3)": "{:.1f}",
     "Gasto N2 (m3/min)": "{:.1f}",
    "Volumen N2 (m3)": "{:.1f}",
}).hide(axis="index"))  # esconder índice del ciclo for

Etapa,Gasto líquido (bpm),Volumen líquido (m3),Gasto N2 (m3/min),Volumen N2 (m3),TP (psi),TR (psi)
1,0.0,5.0,0.0,0.0,640,1363
2,5.0,5.0,40.0,213.3,576,1394
3,5.0,8.0,40.0,429.3,508,1411
4,5.0,4.0,40.0,214.7,422,1371
5,5.0,5.0,40.0,267.3,501,1358
6,5.0,4.0,40.0,214.7,432,1356
7,0.0,0.0,80.0,3197.3,1518,1435
8,3.0,5.0,0.0,0.0,225,1457
9,0.0,1.8,0.0,0.0,8,1464


Descarga el archivo en excel en caso de que quieras copiar y pegar los datos en tu archivo de excel de la gráfica final

In [ ]:
# DESCARGAR ARCHIVO EXCEL

# Guardar
output_name = "Grafica_con_datos.xlsx"
summary.to_excel(output_name, index=False)

# Descargar
files.download(output_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>